# Django Form

In [1]:
from django import forms

class ContentForm(forms.Form):
    subject = forms.CharField()
    email   = forms.EmailField(required=False)
    message = forms.CharField()

# from filter.forms import ContactForm

## views.py에서 Json 만들어서 출력하기

```javascript
options = {
  chart: {
      height: 350,
      type: 'line',
      zoom: { enabled: false}
  },
  dataLabels: { enabled: false },
  stroke: { curve: 'straight'},
  series: [
      { name: "Desktops",
        data: [10, 41, 35, 51, 49, 62, 69, 91, 148]}
  ],
  title: {
      text: 'Trends by Month',
      align: 'left'
  },
  grid: {
      row: {
          colors: ['#f3f3f3', 'transparent'],
          opacity: 0.5
      },
  },
  xaxis: {
      categories: ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep'],
  }
}
```

# Json 객체 Django에서 최대한 가공해서 결과물만 출력합니다
## Sample Data

In [2]:
# pandas 데이터로 line, bar 차트를 출력합니다
import pandas as pd
keys        = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep']
data_value  = [10, 41, 35, 51, 49, 62, 69, 91, 148]
series_data = { key : data_value[no]   for no, key in enumerate(keys)}
series_data = pd.Series(series_data)
series_data.name = 'Desktop'

import random
dataframe_data = pd.DataFrame()
for col in ['col1', 'col2', 'col3']:
    random.shuffle(data_value)
    dataframe_data[col] = data_value

print(type(series_data))
print(type(dataframe_data))
assert type(series_data)    == pd.core.series.Series
assert type(dataframe_data) == pd.core.frame.DataFrame

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


## Functions
Series/ DataFrame 구분하는 함수를 정의합니다

In [3]:
import json
import numpy as np
# pandas series 데이터를 line 데이터로 변환 출력합니다
def apex_line(data, type='line', zoom='False', height=350, marker=0,
              data_label=False, curve='straight', title='None'):

    options = {
        "chart" : {
            "height" : height,
            "type" : type,
            "zoom" : { "enabled" : zoom }
        },
        # 데이터 value 를 포인트에서 출력여부
        "dataLabels" : { "enabled" : data_label },
        "stroke" : { "curve" : curve },
        "title" : {
            "text" : title, # 제목출력
            "align" : 'left'
        },
        "grid" : {
            "row" : {
                "colors" : ['#f3f3f3', 'transparent'],
                "opacity" : 0.5
            },
        },
        "markers": {
            "size": marker,
            "hover": {"sizeOffset": 2}
        },
        "series" : [ # 출력 values
          { "name" : data.name,
            "data" : data.values.tolist() }
        ],
        "xaxis" : {  # 출력 index
          "categories" : data.index.tolist()
        }
    }
    return json.dumps(options)

In [6]:
# 위와 결합해서, 컬럼이 1개 및 Series 면 Single 출력을
# 컬럼이 2 이상이면 DashBoard 타입으로 출력합니다
data_set = ''
def apex_dash(data, type='line', zoom=False, height=350,
              data_label=False, curve='straight', title=None):
    raw_data = data.to_dict(orient='list')
    data_set = []
    for key in raw_data.keys():
        temp_dict = {}
        temp_dict['name'] = key
        temp_dict['data'] = raw_data[key]
        data_set.append(temp_dict)


    options = {
        "chart": {
            "height": height,
            "type": type,
            "zoom": {"enabled": zoom},
        },
        "dataLabels": {"enabled": data_label},
        "stroke": {
            "width": [3, 3, 3],
            "curve": curve,
            "dashArray": [0, 5, 8]
        },
        "title": {
            "text": title,
            "align": 'left'
        },
        "markers": {
            "size": 0,
            "hover": {"sizeOffset": 2}
        },
        "grid": {
            "borderColor": '#f1f1f1',
        },
        "series": data_set, #data_set
        "xaxis": {
            "categories": data.index.tolist(),
        },
    }
    return json.dumps(options)

# Apex Chart Json Exporter
## 위의 2개 함수를 합칩니다
line bar 차트를 출력하는 함수를 정의합니다

In [7]:
import pandas as pd 
import json
def apex_django_json(data, types='line', zoom='False', height=350, marker=0,
              data_label=False, curve='straight', title='None'):

    # data type is Series
    if type(data) == pd.core.series.Series:
        data_set         = {}
        data_set["name"] = data.name
        data_set["data"] = data.values.tolist()
        
    # data type is DataFrame
    elif type(data) == pd.core.frame.DataFrame:    
        raw_data = data.to_dict(orient='list')
        # converting Data
        data_set = []
        for key in raw_data.keys():
            temp_dict         = {}
            temp_dict['name'] = key
            temp_dict['data'] = raw_data[key]
            data_set.append(temp_dict)

    else:
        print('You Need to Convert to Pandas Dataset')
        return None
            
    # Common Elements            
    options = {
        "chart" : {
            "height" : height,
            "type" : types,
            "zoom" : { "enabled" : zoom }
        },
        # Value Point Size & Display the Data on Each Points
        "dataLabels" : { "enabled" : data_label },
        "stroke"     : { "curve" : curve },
        "title" : {
            "text" : title, # Chart Title
            "align" : 'left'
        },
        "grid" : {
            "row" : {
                "colors"  : ['#f3f3f3', 'transparent'],
                "opacity" : 0.5
            },
        },
        "markers" : {
            "size"  : marker,
            "hover" : {"sizeOffset": 2}
        },
        "series": data_set, # Series : Single Dict
        "xaxis" : {
            "categories": data.index.tolist(),
        },
    }
        
    return json.dumps(options)

In [8]:
apex_django_json(dataframe_data)

'{"chart": {"height": 350, "type": "line", "zoom": {"enabled": "False"}}, "dataLabels": {"enabled": false}, "stroke": {"curve": "straight"}, "title": {"text": "None", "align": "left"}, "grid": {"row": {"colors": ["#f3f3f3", "transparent"], "opacity": 0.5}}, "markers": {"size": 0, "hover": {"sizeOffset": 2}}, "series": [{"name": "col1", "data": [41, 49, 69, 62, 10, 35, 51, 91, 148]}, {"name": "col2", "data": [62, 49, 35, 10, 148, 69, 51, 41, 91]}, {"name": "col3", "data": [41, 10, 51, 148, 62, 91, 49, 35, 69]}], "xaxis": {"categories": [0, 1, 2, 3, 4, 5, 6, 7, 8]}}'

# Apex 에서 시계열 데이터 사용하기
시계열을 출력해보자 [StackOverFlow](https://stackoverflow.com/questions/11875770/how-to-overcome-datetime-datetime-not-json-serializable)

```python
Is that Django strips a bit of the data:

 "last_login": "2018-08-03T10:51:42.990", # DjangoJSONEncoder 
 "last_login": "2018-08-03T10:51:42.990239", # default
```

```javascript
dataset = [
    { "x" : new Date(1538778600000),
      "y" : [6629.81, 6650.5, 6623.04, 6633.33]
    },
    { "x" : new Date(1538780400000),
      "y" : [6632.01, 6643.59, 6620, 6630.11]
    }]
```

In [9]:
# Import DataSet
from pandas_datareader import get_data_yahoo
df_raw = get_data_yahoo('005930.Ks', '2018-01-01', '2018-12-31')
df_raw = df_raw.iloc[:, :-1]

# 컬럼 데이터를 Python Int로 변환합니다 
# (Json 변환시 np.int는 불가능 합니다)
for col in df_raw.columns:
    df_raw[col] = df_raw[col].astype(int)
df_raw = df_raw.reset_index()

In [10]:
# DateTime date to Json (포맷에 맞도록 형식을 변형합니다)
from datetime import date, datetime

# 시계열 데이터를 Json 포맷으로 변환하는 함수
def json_serial(obj):
    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError ("Type %s not serializable" % type(obj))

In [11]:
# Json에 맞는 타입으로 시간자료를 바꾸고, 시간을 추가
df_raw.Date = list(map(lambda x : json_serial(x), df_raw.Date))
df_raw.Date = list(map(lambda x : x.replace("T00:00:00", "T15:30:00"), df_raw.Date))

# 필요한 컬럼으로 재정렬한 결과를 출력합니다
df = df_raw[['Date','Open','High','Low','Close']]
df.head(3)

,Date,Open,High,Low,Close
0,2018-01-02T15:30:00,51380,51400,50780,51020
1,2018-01-03T15:30:00,52540,52560,51420,51620
2,2018-01-04T15:30:00,52120,52180,50640,51080


In [12]:
# DataFrame 데이터를 json 포맷으로 변환하기
dataset = []
for no in range(len(df)):
    result = {}
    result["x"] = df.iloc[no, 0]
    result["y"] = list(df.iloc[no, 1:])
    dataset.append(result)

In [13]:
options = {
    "chart": {
        "height": 350,
        "type"  : 'candlestick',
    },
    "title" : {
        "text" : 'CandleStick Chart',
        "align": 'left'
    },
    # 시계열 데이터를 입력합니다
    "xaxis" : {
        "type" : 'datetime'
    },
    "yaxis" : {
        "tooltip" : {
            "enabled": True
        }
    },
    "series" : [{"data": dataset}],
    }

In [14]:
type(options)

dict

# Apex 에서 시계열 데이터 사용하기 2
시계열 데이터와 볼륨을 함께 사용하기

In [19]:
df_raw.head(3)

,Date,High,Low,Open,Close,Volume
0,2018-01-02T15:30:00,51400,50780,51380,51020,8474250
1,2018-01-03T15:30:00,52560,51420,52540,51620,10013500
2,2018-01-04T15:30:00,52180,50640,52120,51080,11695450


In [55]:
# ohlc 데이터 생성하기
raw_data = df_raw.to_dict(orient='split')['data']
seriesData = []
for col_data in raw_data:
    result = {}
    result["x"] = col_data[0]
    result["y"] = col_data[1:-1]
    seriesData.append(result)

seriesData[:3]

[{'x': '2018-01-02T15:30:00', 'y': [51400, 50780, 51380, 51020]},
 {'x': '2018-01-03T15:30:00', 'y': [52560, 51420, 52540, 51620]},
 {'x': '2018-01-04T15:30:00', 'y': [52180, 50640, 52120, 51080]}]

In [56]:
# volume 데이터 생성하기
seriesDataLinear = []
for col_data in raw_data:
    result = {}
    result["x"] = col_data[0]
    result["y"] = col_data[-1]
    seriesDataLinear.append(result)

seriesDataLinear[:3]

[{'x': '2018-01-02T15:30:00', 'y': 8474250},
 {'x': '2018-01-03T15:30:00', 'y': 10013500},
 {'x': '2018-01-04T15:30:00', 'y': 11695450}]

In [57]:
# 캔들차트 그리기 
optionsCandlestick = {
    "chart": {
        "id" : 'candles',
        "height" : 290,
        "type"   : 'candlestick',
        "toolbar": {
            "autoSelected" : 'pan',
            "show" : False
        },
        "zoom" : {"enabled" : False},
    },
    "plotOptions" : {
        "candlestick" : {
            "colors"  : {
            "upward"  : '#3C90EB',
            "downward": '#DF7D46'
            }
        }
    },
    "series" : [{ "data" : seriesData }],
    "xaxis" : { "type" : 'datetime'},
}

In [58]:
# 볼륨차트 그리기
options = {
    "chart" : {
        "height" : 160,
        "type"   : 'bar',
        "brush"  : {
            "enabled" : True,
            "target"  : 'candles'
        },
        "selection" : {
            "enabled" : True,
            "xaxis"   : {
                "min: new Date('20 Jan 2017').getTime()",
                "max: new Date('10 Dec 2017').getTime()"
            },
            "fill"    : {
                "color" : '#ccc',
                "opacity" : 0.4
            },
            "stroke" : {
                "color" : '#0D47A1',
            }
        },
    },
    "dataLabels" : {
        "enabled" : False
    },
    "plotOptions" : {
        "bar" : {
            "columnWidth" : '80%',
            "colors" : {
                "ranges" : [
                    { "from" : -1000,
                      "to" : 0,
                      "color" : '#F15B46'
                    }, {
                        "from" : 1,
                        "to" : 10000,
                        "color" : '#FEB019'
                    }
                ],
            },
        }
    },
    "stroke" : { "width" : 0 },
    "series" : [{
        "name" : 'volume',
        "data" : seriesDataLinear
    }],
    "xaxis" : {
        "type" : 'datetime',
        "axisBorder" : {
            "offsetX" : 13
        }
    },
    "yaxis" : {
        "labels" : {"show" : False}
    }
}

In [68]:
# https://stackoverflow.com/questions/22281059/set-object-is-not-json-serializable
def set_default(obj):
    if isinstance(obj, set):
        return list(obj)
    raise TypeError

# result = json.dumps(yourdata, default=set_default)

In [70]:
# json.dumps(options, default=set_default)

In [71]:
# options
# json.dumps(optionsCandlestick)

In [60]:
def apex_candle_vol(code = '005930.KS', height=350, marker=0):

    # 필요한 컬럼으로 재정렬한 결과를 출력합니다
    df = stock_data('005930.KS')

    # ohlc 데이터 생성하기
    raw_data = df.to_dict(orient='split')['data']
    seriesData = []
    for col_data in raw_data:
        result = {}
        result["x"] = col_data[0]
        result["y"] = col_data[1:-1]
        seriesData.append(result)

    # volume 데이터 생성하기
    seriesDataLinear = []
    for col_data in raw_data:
        result = {}
        result["x"] = col_data[0]
        result["y"] = col_data[-1]
        seriesDataLinear.append(result)

    # 캔들차트 그리기
    optionsCandlestick = {
        "chart": {
            "id": 'candles',
            "height": 290,
            "type": 'candlestick',
            "toolbar": {
                "autoSelected": 'pan',
                "show": False
            },
            "zoom": {"enabled": False},
        },
        "plotOptions": {
            "candlestick": {
                "colors": {
                    "upward": '#3C90EB',
                    "downward": '#DF7D46'
                }
            }
        },
        "series": [{"data": seriesData}],
        "xaxis": {"type": 'datetime'},
    }

    # 볼륨차트 그리기
    options = {
        "chart": {
            "height": 160,
            "type": 'bar',
            "brush": {
                "enabled": True,
                "target": 'candles'
            },
            "selection": {
                "enabled": True,
                "xaxis": {
                    "min: new Date('20 Jan 2017').getTime()",
                    "max: new Date('10 Dec 2017').getTime()"
                },
                "fill": {
                    "color": '#ccc',
                    "opacity": 0.4
                },
                "stroke": {
                    "color": '#0D47A1',
                }
            },
        },
        "dataLabels": {
            "enabled": False
        },
        "plotOptions": {
            "bar": {
                "columnWidth": '80%',
                "colors": {
                    "ranges": [
                        {"from": -1000,
                         "to": 0,
                         "color": '#F15B46'
                         }, {
                            "from": 1,
                            "to": 10000,
                            "color": '#FEB019'
                        }
                    ],
                },
            }
        },
        "stroke": {"width": 0},
        "series": [{
            "name": 'volume',
            "data": seriesDataLinear
        }],
        "xaxis": {
            "type": 'datetime',
            "axisBorder": {
                "offsetX": 13
            }
        },
        "yaxis": {
            "labels": {"show": False}
        }
    }
    return json.dumps(options), json.dumps(optionsCandlestick)


In [72]:
from pandas_datareader import get_data_yahoo
def stock_data(code):
    df = get_data_yahoo(code, '2018-01-01', '2018-12-31')
    df = df.iloc[:, :-1]
    for col in df.columns:
        df[col] = df[col].astype(int)
    df = df.reset_index()

    # Json 타입으로 시간자료를 바꾸고, 시간을 추가
    df.Date = list(map(lambda x: json_serial(x), df.Date))
    df.Date = list(map(lambda x: x.replace("T00:00:00", "T15:30:00"), df.Date))
    return df

In [ ]:
# 데이터 포맷을 변환하는 함수를 작성합니다
from datetime import date, datetime

# 시계열 데이터를 Json 포맷으로 변환하는 함수
def json_serial(obj):
    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError ("Type %s not serializable" % type(obj))

json_serial(datetime.now())

In [ ]:
import pandas as pd
one_year = pd.date_range('2018-01-01',periods=365)
list(map(lambda x: json_serial(x), one_year))[:3]

In [ ]:
# Python 에서 생성된 시계열 값을 Json 타입으로 변환합니다
# from pandas_datareader import get_data_yahoo
# df = get_data_yahoo('005930.Ks', '2018-01-01', '2018-12-31')
# df = df.iloc[:, :-1]
# for col in df.columns:
#     df[col] = df[col].astype(int)
# df = df.reset_index()
# df.to_csv('stock.csv', index=None)

# 데이터 불러오기
df      = pd.read_csv('stock.csv')
df.Date = pd.DatetimeIndex(df.Date)

# Json에 맞는 타입으로 시간자료를 바꾸고, 시간을 추가
df.Date = list(map(lambda x : json_serial(x), df.Date))
df.Date = list(map(lambda x : x.replace("T00:00:00", "T15:30:00"), df.Date))

# 필요한 컬럼으로 재정렬한 결과를 출력합니다
df = df[['Date', 'Open', 'High','Low','Close']]
df.head(3)

In [ ]:
# {'dict', 'list', 'series', 'split', 'records', 'index'}
dataset = []
raw_data = df.to_dict(orient='split')['data']
for col_data in raw_data:
    result = {}
    result["x"] = col_data[0] 
    result["y"] = col_data[1:]
    dataset.append(result)

In [ ]:
dataset[-15:]

In [ ]:
options = {
    "chart": {
        "height": 350,
        "type"  : 'candlestick',
    },
    "title" : {
        "text" : 'CandleStick Chart',
        "align": 'left'
    },
    # 시계열 데이터를 입력합니다
    "xaxis" : {
        "type" : 'datetime'
    },
    "yaxis" : {
        "tooltip" : {
            "enabled": True
        }
    },
    "series" : [{"data": dataset}],
    }


In [ ]:
df.info()

In [ ]:
df.iloc[0,:]

In [ ]:
dataset

In [ ]:
options = {
      "chart": {
        "height" : 350,
        "type"   : 'candlestick',
      },
      "series" : [{
        "data" : dataset
      }],
      "title" : {
        "text"  : 'CandleStick Chart',
        "align" : 'left'
      },
      "xaxis" : {
        "type" : 'datetime'
      },
      "yaxis" : {
        "tooltip" : {
          "enabled" : True
        }
      }
    }